In [1]:
import cv2
import os

putanje_slika = os.listdir ('podaci/train')
print ("[INFO] Učitao ukupno {} slika za treniranje i testiranje".format (len(putanje_slika)))

[INFO] Učitao ukupno 25000 slika za treniranje i testiranje


In [ ]:
podaci = []
oznake = []

def ucitaj_sliku (slika, dimenzije=(400, 300)):
    return cv2.resize (slika, dimenzije)

for i, putanja_slike in enumerate(putanje_slika):
    slika = cv2.imread ('podaci/train/{}'.format(putanja_slike))
    slika = ucitaj_sliku (slika)
    podaci.append (slika)
    
    oznaka = putanja_slike.split(os.path.sep)[-1].split(".")[0]
    oznake.append (oznaka)
    
    if i%1000 == 0:
        print ("[INFO] obrađeno ukupno: {}/{}"
               .format(i, len(putanje_slika)))

[INFO] obrađeno ukupno: 0/25000
[INFO] obrađeno ukupno: 1000/25000
[INFO] obrađeno ukupno: 2000/25000
[INFO] obrađeno ukupno: 3000/25000
[INFO] obrađeno ukupno: 4000/25000
[INFO] obrađeno ukupno: 5000/25000
[INFO] obrađeno ukupno: 6000/25000
[INFO] obrađeno ukupno: 7000/25000
[INFO] obrađeno ukupno: 8000/25000
[INFO] obrađeno ukupno: 9000/25000
[INFO] obrađeno ukupno: 10000/25000
[INFO] obrađeno ukupno: 11000/25000
[INFO] obrađeno ukupno: 12000/25000
[INFO] obrađeno ukupno: 13000/25000
[INFO] obrađeno ukupno: 14000/25000
[INFO] obrađeno ukupno: 15000/25000
[INFO] obrađeno ukupno: 16000/25000
[INFO] obrađeno ukupno: 17000/25000
[INFO] obrađeno ukupno: 18000/25000
[INFO] obrađeno ukupno: 19000/25000
[INFO] obrađeno ukupno: 20000/25000
[INFO] obrađeno ukupno: 21000/25000
[INFO] obrađeno ukupno: 22000/25000
[INFO] obrađeno ukupno: 23000/25000
[INFO] obrađeno ukupno: 24000/25000


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from keras.utils import np_utils

le = LabelEncoder ()
oznake = le.fit_transform (oznake)
oznake = np_utils.to_categorical (oznake, 2)

podaci = np.array (podaci)/255.0

treniranje_podaci, treniranje_oznake, testiranje_podaci, testiranje_oznake=\
train_test_split (podaci, oznake, test_size=0.15, random_state=23)
print ("[INFO] podaci podjeljeni u podatke za treniranje i testiranje.")

Using TensorFlow backend.


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten
from keras.optimizers import SGD

def osnovni_model ():
    model = Sequential()
    model.add (
        Conv2D(32, (3,3), padding='same', 
               input_shape=treniranje_podaci.shape[1:]))
    model.add (Activation('relu'))
    model.add (Conv2d(32, (3,3)))
    model.add (MaxPooling2D(pool_size=(2,2)))
    
    model.add (Conv2D(64, (3,3), padding='same'))
    model.add (Activation ('relu'))
    model.add (Conv2D(64, (3,3)))
    model.add (Activation ('relu'))
    model.add (MaxPooling2D(pool_size=(2,2)))
    
    model.add (Flatten())
    model.add (Dense(512))
    model.add (Activation ('relu'))
    model.add (Dense (len (le.classes_)))
    model.add (Activation ('softmax'))
    
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile (loss='categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])
    return model

In [ ]:
cnn = osnovni_model()
cnn.summary()

In [ ]:
model = cnn.fit (treniranje_podaci, testiranje_podaci, batch_size=32, 
                 epochs=10, validation_data=(testiranje_podaci, testiranje_oznake), 
                 shuffle=True)